In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px

from src.data_utils import get_asx_backtest_data

In [3]:
def load_vix_ts(source = 'csv', ts='3M'):
    if source == 'csv':
        vix_df = pd.read_csv('data/vixcurrent.csv', skiprows=0)
        vix_df.columns = [c.lower() for c in vix_df.columns]
        vix_df.columns = [c.replace('vix ', '') for c in vix_df.columns]
        if ts == '3M':
            vix3m_df = pd.read_csv('data/vix3mdailyprices.csv', skiprows=0)
            vix3m_df.columns = [c.lower() for c in vix3m_df.columns]
            vix3m_df.rename({'unnamed: 0': 'date'}, axis=1, inplace=True)
            vix_df = pd.merge(left = vix_df[['date', 'close']].rename({'close': 'vix'}, axis=1),
                            right = vix3m_df[['date', 'close']].rename({'close': 'vix3m'}, axis=1),
                            on = 'date')
            vix_df['ts'] = vix_df.vix3m - vix_df.vix
        elif ts == '9D':
            vix9d_df = pd.read_csv('data/VIX9D_History.csv', skiprows=0)
            vix9d_df.columns = [c.lower() for c in vix9d_df.columns]
            vix_df = pd.merge(left = vix_df[['date', 'close']].rename({'close': 'vix'}, axis=1),
                            right = vix9d_df[['date', 'close']].rename({'close': 'vix9d'}, axis=1),
                            on = 'date')
            vix_df['ts'] = vix_df.vix - vix_df.vix9d
        vix_df['date'] = vix_df.date.apply(datetime.strptime, args=['%m/%d/%Y'])
    elif source == 'intraday':
        vix_dir = '../backtesting/Volatility/'
        ts_time = 44
        bars = 5
        vix_fn = f'data/vix_intraday_ts_tst{ts_time}_bars{bars}.csv'
        vix_df = pd.read_csv(vix_dir + vix_fn)
        vix_df['date'] = vix_df.short_date.apply(datetime.strptime, args=['%Y-%m-%d'])
    return(vix_df)


from src.get_daily_data_av import get_daily_data


def get_backtest_data(long_assets, bond_assets, attempts):
    long_price_df = get_daily_data(long_assets, attempts)[0]
    bond_price_df = pd.DataFrame()#get_daily_data(bond_assets, attempts)[0]
    price_df = pd.concat([long_price_df, bond_price_df])
    price_df['ret'] = price_df.groupby('symbol').adjusted_close.pct_change()
    price_df['ret'] = price_df.groupby('symbol').ret.shift(-1)
    return(price_df)

In [4]:
vix_df = load_vix_ts('intraday')
vix_df['symbol'] = 'ts'
# print(vix_df.columns)
vix_df = vix_df[['date', 'ts', 'close', 'close3m']].rename({'close': 'vix', 'close3m': 'vix3m'}, axis=1)
vix_df.sort_values('date', inplace=True)#.tail()

In [5]:
long_assets = ['VBK', 'SPY', 'VTI', 'QQQ', 'IJR', 'IJH', 'IWM', 'DIA'] #L1
bond_assets = []
attempts = 0

price_df = get_backtest_data(long_assets, bond_assets, attempts)

fetching 1/8 symbol VBK
['VBK.csv']
attempting to read data from disk
fetching 2/8 symbol SPY
['SPY.csv']
attempting to read data from disk
fetching 3/8 symbol VTI
['VTI.csv']
attempting to read data from disk
fetching 4/8 symbol QQQ
['QQQ.csv']
attempting to read data from disk
fetching 5/8 symbol IJR
['IJR.csv']
attempting to read data from disk
fetching 6/8 symbol IJH
[]
fetching 7/8 symbol IWM
['IWM.csv']
attempting to read data from disk
fetching 8/8 symbol DIA
['DIA.csv']
attempting to read data from disk
adding date information
filling forward prices
adjusting prices


In [6]:
price_df.head()

,date,low,open,high,close,volume,adjusted_close,split_coefficient,dividend_amount,symbol,adjusted_ratio,adjusted_open,adjusted_high,adjusted_low,ret
0,2004-01-30,48.56,48.56,49.00,49.00,2100.0,43.588325,1.0,0.0,VBK,0.889558,43.196919,43.588325,43.196919,-0.001020
1,2004-02-02,48.95,49.15,49.15,48.95,1300.0,43.543847,1.0,0.0,VBK,0.889558,43.721758,43.721758,43.543847,-0.002860
2,2004-02-03,48.81,48.90,48.90,48.81,800.0,43.419309,1.0,0.0,VBK,0.889558,43.499369,43.499369,43.419309,-0.021717
3,2004-02-04,47.75,48.40,48.40,47.75,1000.0,42.476378,1.0,0.0,VBK,0.889558,43.054590,43.054590,42.476378,0.003141
4,2004-02-05,47.90,48.20,48.20,47.90,500.0,42.609811,1.0,0.0,VBK,0.889558,42.876679,42.876679,42.609811,0.024008


In [7]:
price_df['date'] = pd.to_datetime(price_df.date)

In [8]:
# px.line(price_df.loc[price_df.symbol == 'VBK'], x = 'date', y='adjusted_close')

In [9]:
# join vix

universe_df = pd.merge(left = price_df.copy(),
        left_on = 'date',
        right = vix_df,
        right_on = 'date', 
        how = 'left')

universe_df.sort_values(['symbol', 'date'], inplace=True)

In [10]:
# get batches

In [11]:
def form_batches(price_df, lb_win=10, train_cols=None, use_vix=False):
    
    n_channels = 3
    if use_vix:
        n_channels=5
        price_df.vix.ffill(inplace=True)
        price_df.ts.ffill(inplace=True)
    
    price_df['log_close_lag'] = np.log(price_df.groupby('symbol').adjusted_close.shift(1))
#     price_df['o_ret'] = np.log(price_df.adj_open) - price_df.log_close_lag
#     price_df['c_ret'] = np.log(price_df.adj_close) - price_df.log_close_lag
#     price_df['target'] = price_df.groupby('symbol').c_ret.shift(-1)
#     price_df['target'] = np.sign(price_df.target)

    log_close = np.log(price_df.adjusted_close)
    price_df['o_ret'] = np.log(price_df.adjusted_open) - price_df.log_close_lag
    price_df['hlvol'] = np.log(price_df.high) - np.log(price_df.low) / price_df.open
    price_df['ret'] = price_df.groupby('symbol').adjusted_close.pct_change()
    price_df['ret'] = price_df.groupby('symbol').ret.shift(-1)
    price_df['target'] = (price_df.ret.values > 0).astype(int)

    cno_list = []
    cnv_list=[]
    cnc_list=[]
    vix_list=[]
    ts_list=[]
    for i in range(lb_win):
        i += 1
        log_close_lag = np.log(price_df.groupby('symbol').adjusted_close.shift(i))
        cnc = f'c_ret_l{i-1}'
        price_df[cnc] = log_close - log_close_lag
        cno = f'o_ret_l{i-1}'
        price_df[cno] = price_df.groupby('symbol').o_ret.shift(i-1)
        cnv = f'hlvol_l{i-1}'
        price_df[cnv] = price_df.groupby('symbol').hlvol.shift(i-1)
        cnc_list.append(cnc)
        cno_list.append(cno)
        cnv_list.append(cnv)
        if use_vix:
            vix = f'vix_l{i}'
            price_df[vix] = price_df.groupby('symbol').vix.shift(i)
            ts = f'ts_l{i}'
            price_df[ts] = price_df.groupby('symbol').ts.shift(i)
            vix_list.append(vix)
            ts_list.append(ts)

    if not train_cols:
        train_cols = [item for sublist in [cnc_list, cno_list, cnv_list, vix_list, ts_list] for item in sublist]# + ['VIX', 'ts']
    batch_df = price_df[['date', 'symbol', 'target', 'ret'] + train_cols].dropna().copy()
    
    n_batches = batch_df.shape[0]
    X = batch_df[train_cols].values.reshape(n_batches, n_channels, lb_win)
    X = np.array([b.transpose() for b in X])
    
    y = batch_df.target.values
    
    return X, y, train_cols, batch_df.index, price_df

In [110]:
####
lb_win = 10
use_vix = True
split_date = '2019-01-01'
last_train_date = '2018-12-01'
time_splits = [universe_df.loc[universe_df.date < last_train_date].index.values,
               universe_df.loc[universe_df.date >= split_date].index.values]

X_train, y_train, train_cols, train_index, train_price_df =\
            form_batches(universe_df.loc[time_splits[0]].copy(), lb_win, use_vix=use_vix)
X_test, y_test, _, test_index, test_price_df =\
            form_batches(universe_df.loc[time_splits[1]].copy(), lb_win, train_cols, use_vix=use_vix)

<ipython-input-11-5bb775bbd25f>:20: RuntimeWarning:

invalid value encountered in greater

<ipython-input-11-5bb775bbd25f>:20: RuntimeWarning:

invalid value encountered in greater



In [111]:
train_price_df[['date', 'adjusted_close', 'c_ret_l1', 'c_ret_l2', 'c_ret_l3', 'ret', 'target', 'ts']].tail()#.dropna()

,date,adjusted_close,c_ret_l1,c_ret_l2,c_ret_l3,ret,target,ts
14230,2018-11-26,130.710757,0.010033,0.014903,-0.003056,0.001239,1,1.184
14231,2018-11-27,130.872681,0.016659,0.011271,0.016141,0.023290,1,1.002
14232,2018-11-28,133.920662,0.024261,0.039682,0.034294,-0.001849,0,0.796
14233,2018-11-29,133.673014,0.021172,0.022410,0.037831,0.007197,1,0.776
14234,2018-11-30,134.635033,0.005320,0.028343,0.029581,NaN,0,0.680


In [112]:
train_price_df[['date', 'log_close_lag', 'adjusted_open', 'o_ret_l1', 'o_ret_l2', 'o_ret_l3', 'ret']].tail()#.dropna()

,date,log_close_lag,adjusted_open,o_ret_l1,o_ret_l2,o_ret_l3,ret
14230,2018-11-26,4.857566,129.996387,-0.006943,0.005900,-0.014047,0.001239
14231,2018-11-27,4.872987,130.158311,0.009941,-0.006943,0.005900,0.023290
14232,2018-11-28,4.874225,131.472753,-0.004235,0.009941,-0.006943,-0.001849
14233,2018-11-29,4.897248,133.530140,0.004575,-0.004235,0.009941,0.007197
14234,2018-11-30,4.895397,133.577764,-0.002920,0.004575,-0.004235,NaN


In [113]:
# Multi rocket

In [114]:
import getopt
import os
import platform
import socket
import sys
import time
from datetime import datetime

import numba
import numpy as np
import pandas as pd
import psutil
import pytz
from sklearn.metrics import accuracy_score
from sktime.utils.data_io import load_from_tsfile_to_dataframe

from multirocket.multirocket import MultiRocket
from utils.data_loader import process_ts_data
from utils.tools import create_directory

pd.set_option('display.max_columns', 500)

data_path = "data/sample_mtsc/"
problem = "BasicMotions"
itr = 0
kernel_selection = 0
num_features = 8000
feature_id = 202
save = True
num_threads = 0

In [83]:
if kernel_selection == 0:
    # for now, change type to float32. will standardise in future.
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # using minirocket_multivariate, so need 3 shapes (n_instances, channel, time)
    # X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]))
    # X_test = X_test.reshape((X_test.shape[0], X_test.shape[1]))

nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

classifier = MultiRocket(num_features=num_features,
                         feature_id=feature_id,
                         kernel_selection=kernel_selection)
yhat_train = classifier.fit(X_train, y_train,
                            predict_on_train=False)
if yhat_train is not None:
    train_acc = accuracy_score(y_train, yhat_train)
else:
    train_acc = -1

# yhat_test = classifier.predict(X_test)
# test_acc = accuracy_score(y_test, yhat_test)


FeatureID: 202 -- features for each kernel: ['PPV', 'CO_HistogramAMI_even_2_5', 'SB_BinaryStats_mean_longstretch1', 'SB_MotifThree_quantile_hh', 'LongStretch_above_0', 'Max']
Creating MiniRocket_202_8000 with 4000 kernels
Using time series split
[MiniRocket_202_8000] Training with training set of (29396, 10, 5)
Kernels applied!, took 52.8149104289987s
Transformed Shape (29396, 7896)
Training


/home/aptperson/miniconda3/envs/aws_py38/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:187: LinAlgWarning:

Ill-conditioned matrix (rcond=2.56255e-08): result may not be accurate.



Training done!, took 443.447s


In [84]:
def insert_preds(model, X, y, df, index):

    yhat = classifier.predict_proba(X)

    yhat_df = pd.DataFrame([y, yhat.argmax(axis=1), yhat[:, 0], yhat[:, 1]]).T
    yhat_df.columns = ['y', 'yhat', 'p0', 'p1']
    yhat_df.index=index

    out_df = pd.merge(left = df,
                       left_index=True,
                       right = yhat_df,
                       right_index=True,
                       how = 'inner')
    return out_df.sort_values('date')

In [85]:
yhat_train_df = insert_preds(classifier, X_train, y_train, universe_df, train_index.values)

[MiniRocket_202_8000] Predicting
Kernels applied!, took 58.362s. Transformed shape: (29396, 7896). 
[MiniRocket_202_8000] Predicting completed, took 59.819s


In [86]:
print(yhat_train_df[['p0', 'p1']].describe())
print(yhat_train_df.groupby(['yhat', 'y']).size())
mask = yhat_train_df.yhat == 1.0
(yhat_train_df.loc[mask].y == 1).mean()

                 p0            p1
count  29396.000000  29396.000000
mean       0.481603      0.518397
std        0.103388      0.103388
min        0.162866      0.140160
25%        0.413857      0.452820
50%        0.480820      0.519180
75%        0.547180      0.586143
max        0.859840      0.837134
yhat  y  
0.0   0.0     9097
      1.0     3312
1.0   0.0     4477
      1.0    12510
dtype: int64


0.7364455171601814

In [87]:
yhat_test_df = insert_preds(classifier, X_test, y_test, universe_df, test_index.values)

[MiniRocket_202_8000] Predicting
Kernels applied!, took 6.894s. Transformed shape: (4305, 7896). 
[MiniRocket_202_8000] Predicting completed, took 7.112s


In [88]:
print(yhat_test_df[['p0', 'p1']].describe())
print(yhat_test_df.groupby(['yhat', 'y']).size())
mask = yhat_test_df.yhat == 1.0
(yhat_test_df.loc[mask].y == 1).mean()

                p0           p1
count  4305.000000  4305.000000
mean      0.521449     0.478551
std       0.160085     0.160085
min       0.016257     0.003299
25%       0.417841     0.379632
50%       0.513263     0.486737
75%       0.620368     0.582159
max       0.996701     0.983743
yhat  y  
0.0   0.0     999
      1.0    1316
1.0   0.0     854
      1.0    1136
dtype: int64


0.5708542713567839

In [89]:
thresh = 0.55
tmp = yhat_test_df.loc[yhat_test_df.p0 > thresh]
print(f'N {tmp.shape[0]}/{(yhat_test_df.p0 > 0.5).sum()}, {(tmp.y == 0).mean()}')

N 1758/2315, 0.4243458475540387


In [90]:
thresh = 0.55
tmp = yhat_test_df.loc[yhat_test_df.p1 > thresh]
print(f'N {tmp.shape[0]}/{(yhat_test_df.p1 > 0.5).sum()}, {(tmp.y == 1).mean()}')

N 1393/1990, 0.5764536970567121


In [107]:
test_acc_df = yhat_test_df.groupby('date')[['y', 'yhat']].apply(lambda x : (x.y == x.yhat).mean()).to_frame('acc')
test_acc_df['roll_acc'] = test_acc_df.acc.cumsum() / (np.arange(test_acc_df.shape[0]) + 1)
test_acc_df['H'] = np.arange(test_acc_df.shape[0]) + 1
px.line(test_acc_df.reset_index(), x='H', y='roll_acc')

In [109]:
test_acc_df = yhat_train_df.groupby('date')[['y', 'yhat']].apply(lambda x : (x.y == x.yhat).mean()).to_frame('acc')
test_acc_df['roll_acc'] = test_acc_df.acc.cumsum() / (np.arange(test_acc_df.shape[0]) + 1)
test_acc_df['H'] = np.arange(test_acc_df.shape[0]) + 1
px.line(test_acc_df.reset_index(), x='H', y='roll_acc')

In [91]:
def performance_metrics(pnl_df, pnl_col = 'ret', N=252, plot=True):
    ret = pnl_df[pnl_col]
    sr = ret.mean() / ret.std() * np.sqrt(N)
    print(f'sr {sr}')
    print(f'kl {ret.mean() / ret.var()}')
    print(f'vol {ret.std() * np.sqrt(N)}')
    cagr = calc_cagr(pnl_df, verbose = 1)
    fig_dd, mdd = calc_draw_down(pnl_df, plot=plot)
    if plot:
        fig = px.line(pnl_df, x='date', y='cpnl', log_y=True)
        fig.show()
        fig_dd.show()
    return(sr, cagr, mdd)
        
        
def calc_cagr(pnl_df, verbose=1):
    cagr = pnl_df[['date', 'cpnl']].dropna().copy()
    t = get_date_range(cagr)
    cagr = ((cagr.cpnl.values[-1]) ** (1/t) - 1) * 100
    if verbose > 0:
        print(f'years {t: .1f}')
        print(f'CAGR {cagr: .2f}')
    # else:
    return(cagr)
    
        
def get_date_range(df):
    start_date = df.date.min()
    end_date = df.date.max()
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    t = end_date - start_date
    return(t.days / 365.25)
        

def calc_draw_down(pnl_df, leverage=None, plot=True):
    if leverage is not None:
        pnl_df['high_water_l'] = pnl_df.equity_l.cummax()
        pnl_df['dd_l'] = -(1 - (pnl_df.equity_l / pnl_df.high_water_l))
        mdd = -pnl_df.dd_l.min()
        print(f'Max DD {-pnl_df.dd_l.min(): .3f}')
    else:
        pnl_df['high_water'] = pnl_df.cpnl.cummax()
        pnl_df['dd'] = -(1 - (pnl_df.cpnl / pnl_df.high_water))
        mdd = -pnl_df.dd.min()
        print(f'Max DD {mdd: .3f}')
    if plot:
        fig = px.line(pnl_df, x='date', y='dd')
    else:
        fig=None
    return(fig, mdd)

In [96]:
N = 2
mask = yhat_train_df.p1 > 0.55 # & batch_df.symbol.isin(train_sym)
yhat_train_df['ret'] = yhat_train_df.groupby('symbol').adjusted_close.pct_change()
yhat_train_df['ret'] = yhat_train_df.groupby('symbol').ret.shift(-1)

pnl_df = yhat_train_df.loc[mask].sort_values('p1').groupby('date').tail(N)#.ret.mean()
max_train_prob = pnl_df.p1.max()
pnl_df['ret'] = pnl_df.ret * (pnl_df.p1/max_train_prob)# * 3
pnl_df = pnl_df.groupby('date').ret.mean().to_frame('ret')
pnl_df.sort_values('date', inplace=True)
pnl_df['cpnl'] = (1 + pnl_df.ret).cumprod()
performance_metrics(pnl_df.reset_index())

sr 6.515364583407245
kl 41.826936942660915
vol 0.15576958437905533
years  19.1
CAGR  87.76
Max DD  0.092


(6.515364583407245, 87.76185502809071, 0.09202827001245417)

In [95]:
N = 2
mask = yhat_test_df.p1 > 0.55 # & batch_df.symbol.isin(train_sym)
yhat_test_df['ret'] = yhat_test_df.groupby('symbol').adjusted_close.pct_change()
yhat_test_df['ret'] = yhat_test_df.groupby('symbol').ret.shift(-1)

pnl_df = yhat_test_df.loc[mask].sort_values('p1').groupby('date').tail(N)#.ret.mean()
max_train_prob = pnl_df.p1.max()
pnl_df['ret'] = pnl_df.ret * (pnl_df.p1/max_train_prob)# * 3
pnl_df = pnl_df.groupby('date').ret.mean().to_frame('ret')
pnl_df.sort_values('date', inplace=True)
pnl_df['cpnl'] = (1 + pnl_df.ret).cumprod()
performance_metrics(pnl_df.reset_index())

sr 0.8484881199563672
kl 5.082033432992899
vol 0.1669583900113537
years  2.4
CAGR  7.15
Max DD  0.085


(0.8484881199563672, 7.153125378699432, 0.08533454655512707)

In [28]:
N = 3
mask = test_df.p1 > 0.5 # & batch_df.symbol.isin(train_sym)
test_df['ret'] = test_df.groupby('symbol').adjusted_close.pct_change()
test_df['ret'] = test_df.groupby('symbol').ret.shift(-1)

pnl_df = test_df.loc[mask].sort_values('p1').groupby('date').tail(N)#.ret.mean()
max_train_prob = pnl_df.p1.max()
pnl_df['ret'] = pnl_df.ret * (pnl_df.p1/max_train_prob)# * 3
pnl_df = pnl_df.groupby('date').ret.mean().to_frame('ret')
pnl_df.sort_values('date', inplace=True)
pnl_df['cpnl'] = (1 + pnl_df.ret).cumprod()
performance_metrics(pnl_df.reset_index())

NameError: name 'test_df' is not defined

In [ ]:
N = 1
pnl_df = test_df.sort_values('p0').groupby('date').tail(N)#.ret.mean()
max_train_prob = pnl_df.p0.max()
pnl_df['ret'] = -pnl_df.ret * (pnl_df.p0/max_train_prob)# * 3
pnl_df = pnl_df.groupby('date').ret.mean().to_frame('ret')
pnl_df['cpnl'] = (1 + pnl_df.ret).cumprod()
performance_metrics(pnl_df.reset_index())

In [ ]:
pnl_df = test_df#.sort_values('p0').groupby('date').tail(N)#.ret.mean()
max_train_prob = pnl_df.p0.max()
# pnl_df['ret'] = pnl_df.ret * (pnl_df.p0/max_train_prob)# * 3
pnl_df = pnl_df.groupby('date').ret.mean().to_frame('ret')
pnl_df['cpnl'] = (1 + pnl_df.ret).cumprod()
performance_metrics(pnl_df.reset_index())